# Concierge Agent using Google ADK

This notebook implements a comprehensive Concierge Agent using **Google's Agent Development Kit (ADK)**. 
It follows a **Multi-Agent Architecture** where a Root Coordinator delegates tasks to specialized sub-agents.

## Architecture
- **Root Coordinator**: Orchestrates the workflow.
- **Planning Agent**: Handles trip suggestions, itineraries, and activities.
- **Booking Agent**: Handles flights, hotels, rides, and bookings.
- **Utility Agent**: Handles weather, currency, visa, insurance, and real-time updates (Mock data).
- **Search Agent**: Handles real-time information retrieval using Google Search (Currency, Weather, Events).
- **Social Agent**: Handles user preferences, feedback, and social media sharing.

## Features Implemented (15/15):
1. Trip Planning, 2. Flight & Hotel Booking, 3. Transportation, 4. Activities, 5. Real-Time Updates,
6. Documents, 7. Budget, 8. Personalized Support, 9. Emergency, 10. Feedback,
11. Insurance, 12. Translation, 13. Currency, 14. Weather, 15. Social Media

In [ ]:
# Install ADK if not already installed (Uncomment if needed)
# !pip install google-adk


In [1]:
import os
import json
import asyncio
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    print("Warning: GOOGLE_API_KEY not found in environment. Please set it.")
    # os.environ["GOOGLE_API_KEY"] = "YOUR_KEY_HERE"
else:
    print("✅ Google API Key loaded.")


✅ Google API Key loaded.


In [2]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


## 1. Define Tools (Mock Implementations)
We define the 15 capabilities as standard Python functions.

In [3]:
# --- Planning Tools ---
def suggest_destinations(budget: str, season: str, interests: str):
    """Suggests travel destinations based on user preferences."""
    return json.dumps([
        {"city": "Kyoto", "country": "Japan", "reason": "Autumn foliage", "cost": "Medium"},
        {"city": "Reykjavik", "country": "Iceland", "reason": "Northern lights", "cost": "High"},
        {"city": "Bali", "country": "Indonesia", "reason": "Beaches", "cost": "Low"}
    ])

def create_itinerary(destination: str, days: int):
    """Creates a day-by-day itinerary."""
    return json.dumps({f"Day {i}": f"Activity in {destination}" for i in range(1, days + 1)})

def suggest_activities(city: str, interests: str):
    """Suggests activities in a city."""
    return json.dumps(["Museum Tour", "Hiking", "Food Tasting"])

# --- Booking Tools ---
def search_flights(origin: str, destination: str, date: str):
    """Searches for flights."""
    return json.dumps([{"airline": "AirFly", "price": 500, "id": "FL123"}])

def book_flight(flight_id: str, passenger_name: str):
    """Books a flight."""
    return json.dumps({"status": "confirmed", "ref": "BK-FL-001"})

def search_hotels(city: str, check_in: str):
    """Searches for hotels."""
    return json.dumps([{"name": "Grand Hotel", "price": 200, "id": "HTL1"}])

def book_hotel(hotel_id: str, guest_name: str):
    """Books a hotel."""
    return json.dumps({"status": "confirmed", "ref": "BK-HTL-001"})

def book_ride(pickup: str, dropoff: str):
    """Books a local ride."""
    return json.dumps({"driver": "John", "eta": "5 mins"})

def book_activity(activity_name: str, date: str):
    """Books an activity ticket."""
    return json.dumps({"status": "confirmed", "ticket": "ACT-001"})

# --- Utility Tools ---
def get_weather_forecast(city: str):
    """Gets weather forecast."""
    return "Sunny, 25°C"

def convert_currency(amount: float, from_curr: str, to_curr: str):
    """Converts currency."""
    return f"{amount * 1.1:.2f} {to_curr}"

def translate_text(text: str, target_lang: str):
    """Translates text."""
    return f"[Translated to {target_lang}]: {text}"

def check_visa_requirements(citizenship: str, country: str):
    """Checks visa requirements."""
    return "Visa-free for 90 days (Simulated)"

def get_insurance_quote(destination: str, days: int):
    """Gets travel insurance quote."""
    return "$50 Standard Plan"

def get_emergency_contacts(city: str):
    """Gets emergency contacts."""
    return "Police: 911, Embassy: +1-555-0199"

def get_flight_status(flight_number: str):
    """Checks flight status."""
    return "On Time"

def track_expense(item: str, amount: float):
    """Logs an expense."""
    return "Expense logged."

def get_budget_summary():
    """Returns total expenses."""
    return "Total: $150"

# --- Social Tools ---
user_prefs = {}
def update_user_preference(key: str, value: str):
    """Updates user preference."""
    user_prefs[key] = value
    return "Updated."

def get_user_preferences():
    """Gets user preferences."""
    return json.dumps(user_prefs)

def submit_feedback(rating: int, comment: str):
    """Submits feedback."""
    return "Feedback received."

def share_to_social_media(platform: str, content: str):
    """Shares content to social media."""
    return f"Shared to {platform}."


In [4]:
# --- Helper Utilities for Session and Metrics ---



# Simple Session Service (in-memory)

class SimpleSessionService:

    def __init__(self):

        self.sessions = {}

    def get(self, session_id):

        return self.sessions.get(session_id, {})

    def set(self, session_id, state):

        self.sessions[session_id] = state

    def clear(self, session_id):

        self.sessions.pop(session_id, None)



session_service = SimpleSessionService()



# Context Compaction (simple token limit)

def compact_context(messages, max_tokens=500):

    """Very naive compaction: keep last N messages"""

    if len(messages) <= max_tokens:

        return messages

    return messages[-max_tokens:]



# Metrics counters

class Metrics:

    def __init__(self):

        self.counters = {}

    def inc(self, name, amount=1):

        self.counters[name] = self.counters.get(name, 0) + amount

    def report(self):

        return self.counters



metrics = Metrics()



print("✅ Helper utilities initialized (session_service, metrics, compact_context)")

✅ Helper utilities initialized (session_service, metrics, compact_context)


## 2. Define Sub-Agents
We group the tools into specialized agents.

In [5]:
# Planning Agent
planning_agent = Agent(
    name="PlanningAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Travel Planner. Use your tools to suggest destinations, create itineraries, and suggest activities.
    IMPORTANT: After using a tool, you MUST provide a text summary of the results to the user. Do not just return the tool output.""",
    tools=[FunctionTool(suggest_destinations), FunctionTool(create_itinerary), FunctionTool(suggest_activities)],
    output_key="planning_output"
)

# Booking Agent
booking_agent = Agent(
    name="BookingAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Booking Specialist. Use your tools to search and book flights, hotels, rides, and activities.
    IMPORTANT: After using a tool, you MUST provide a text confirmation or summary of the booking details to the user.""",
    tools=[
        FunctionTool(search_flights), FunctionTool(book_flight),
        FunctionTool(search_hotels), FunctionTool(book_hotel),
        FunctionTool(book_ride), FunctionTool(book_activity)
    ],
    output_key="booking_output"
)

# Utility Agent (Mock Tools)
utility_agent = Agent(
    name="UtilityAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Travel Assistant. Provide info on weather, currency, visa, insurance, emergency contacts, and flight status using your specific tools.
    IMPORTANT: After using a tool, you MUST provide a clear text report of the information to the user.""",
    tools=[
        FunctionTool(get_weather_forecast), FunctionTool(convert_currency),
        FunctionTool(translate_text), FunctionTool(check_visa_requirements),
        FunctionTool(get_insurance_quote), FunctionTool(get_emergency_contacts),
        FunctionTool(get_flight_status), FunctionTool(track_expense),
        FunctionTool(get_budget_summary)
    ],
    output_key="utility_output"
)

# Search Agent (Real-time Tools)
# We separate this to avoid mixing FunctionTool and google_search which causes compatibility issues.
search_agent = Agent(
    name="SearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Real-Time Information Specialist. Use `google_search` to find up-to-date information on:
    - Currency exchange rates
    - Real-time weather conditions
    - Visa requirements and travel advisories
    - Local events and news
    Always summarize the search results clearly for the user.""",
    tools=[google_search],
    output_key="search_output"
)

# Social Agent
social_agent = Agent(
    name="SocialAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You handle User Profile and Socials. Update preferences, collect feedback, and share updates.
    IMPORTANT: After using a tool, you MUST provide a text confirmation of the action to the user.""",
    tools=[
        FunctionTool(update_user_preference), FunctionTool(get_user_preferences),
        FunctionTool(submit_feedback), FunctionTool(share_to_social_media)
    ],
    output_key="social_output"
)

print("✅ Sub-agents created. SearchAgent separated for compatibility.")


✅ Sub-agents created. SearchAgent separated for compatibility.


## 3. Define Root Coordinator Agent
This agent orchestrates the others.

In [6]:
root_agent = Agent(
    name="ConciergeCoordinator",
    model="gemini-2.5-flash-lite",
    instruction="""You are the Head Concierge. Your goal is to assist the user with their travel needs by coordinating with specialized agents.
    - For planning (destinations, itineraries), call `PlanningAgent`.
    - For bookings (flights, hotels, rides), call `BookingAgent`.
    - For specific utilities (simulated weather, simulated currency, expenses), call `UtilityAgent`.
    - For REAL-TIME information (actual currency rates, actual weather, news), call `SearchAgent`.
    - For social/profile (preferences, feedback, sharing), call `SocialAgent`.
    
    Always answer the user politely. If a sub-agent returns information, summarize it for the user and ask if they need anything else.""",
    tools=[
        AgentTool(planning_agent),
        AgentTool(booking_agent),
        AgentTool(utility_agent),
        AgentTool(search_agent),
        AgentTool(social_agent)
    ]
)

print("✅ Root Coordinator created.")


✅ Root Coordinator created.


## 4. Run the Agent (Conversational Mode)
We use `InMemoryRunner` to execute a conversational loop.

In [ ]:
# --- Missing Feature Implementations ---
# 1. Mock MCP Tool (Multi-Component Process)
class MCPTool(FunctionTool):
    def __init__(self, name: str):
        super().__init__(self.run)
        self.name = name
    def run(self, *args, **kwargs):
        # Simulate a multi-step process
        print(f'MCP {self.name} called with args={args}, kwargs={kwargs}')
        return f'MCP {self.name} completed'

# 2. OpenAPI Tool (mock)
def call_openapi(endpoint: str, payload: dict):
    print(f'OpenAPI Call to endpoint={endpoint}')
    # In a real scenario you would use requests.post...
    return {'status': 'success', 'data': payload}

# 3. Simple Session Service (in-memory)
class SimpleSessionService:
    def __init__(self):
        self.sessions = {}
    def get(self, session_id):
        return self.sessions.get(session_id, {})
    def set(self, session_id, state):
        self.sessions[session_id] = state
    def clear(self, session_id):
        self.sessions.pop(session_id, None)

session_service = SimpleSessionService()

# 4. Context Compaction (simple token limit)
def compact_context(messages, max_tokens=500):
    # Very naive compaction: keep last N messages
    if len(messages) <= max_tokens:
        return messages
    return messages[-max_tokens:]

# 5. Metrics (simple counters)
class Metrics:
    def __init__(self):
        self.counters = {}
    def inc(self, name, amount=1):
        self.counters[name] = self.counters.get(name, 0) + amount
    def report(self):
        return self.counters

metrics = Metrics()

# 6. Agent Evaluation (simple scoring)
def evaluate_response(response: str) -> float:
    # Placeholder: reward length and presence of keywords
    score = len(response) * 0.01
    for kw in ['success', 'confirmed', 'done']:
        if kw in response.lower():
            score += 0.5
    return min(score, 1.0)

# 7. Register new tools with agents
mcp_tool = MCPTool('example_mcp')
openapi_tool = FunctionTool(lambda endpoint, payload: call_openapi(endpoint, payload))



✅ Missing feature implementations added.


In [8]:
# --- Long‑running operation helpers (pause / resume) ---
import asyncio

# Simple flag‑based pause/resume for agents
class AgentPauseController:
    def __init__(self):
        self.paused = False
    async def pause(self):
        self.paused = True
        print('🛑 Agent execution paused')
    async def resume(self):
        self.paused = False
        print('▶️ Agent execution resumed')
    async def wait_if_paused(self):
        while self.paused:
            await asyncio.sleep(0.5)

# Create a global controller that can be used by any agent
agent_pause_controller = AgentPauseController()

# Expose as FunctionTool so agents can request pause/resume
pause_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.pause()))
resume_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.resume()))
# You can add these tools to any agent's tool list if needed
# Example: planning_agent.tools.append(pause_tool)
#          planning_agent.tools.append(resume_tool)
print('✅ Pause/Resume utilities added')


✅ Pause/Resume utilities added


In [9]:
# --- Long‑running operation helpers (pause / resume) ---
import asyncio

# Simple flag‑based pause/resume for agents
class AgentPauseController:
    def __init__(self):
        self.paused = False
    async def pause(self):
        self.paused = True
        print('🛑 Agent execution paused')
    async def resume(self):
        self.paused = False
        print('▶️ Agent execution resumed')
    async def wait_if_paused(self):
        while self.paused:
            await asyncio.sleep(0.5)

# Create a global controller that can be used by any agent
agent_pause_controller = AgentPauseController()

# Expose as FunctionTool so agents can request pause/resume
pause_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.pause()))
resume_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.resume()))
# You can add these tools to any agent's tool list if needed
# Example: planning_agent.tools.append(pause_tool)
#          planning_agent.tools.append(resume_tool)
print('✅ Pause/Resume utilities added')


✅ Pause/Resume utilities added


## MCP (Model Context Protocol) Implementation

This section implements a complete MCP server that exposes all 23 concierge tools.

**Benefits:**
- Standardized protocol for AI tools
- Better interoperability
- Production-ready deployment


In [10]:
# MCP Server - Define tool handlers
from mcp.server import Server
from mcp.types import Tool, TextContent
import mcp.server.stdio

# Create server instance
mcp_server = Server("concierge-agent")

print("✅ MCP Server instance created")


✅ MCP Server instance created


In [11]:
@mcp_server.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    """Handle tool calls from MCP clients"""
    
    # Planning Tools
    if name == "suggest_destinations":
        result = suggest_destinations(arguments.get("budget", ""), arguments.get("season", ""), arguments.get("interests", ""))
        return [TextContent(type="text", text=result)]
    elif name == "create_itinerary":
        result = create_itinerary(arguments.get("destination", ""), arguments.get("days", 1))
        return [TextContent(type="text", text=result)]
    elif name == "suggest_activities":
        result = suggest_activities(arguments.get("city", ""), arguments.get("interests", ""))
        return [TextContent(type="text", text=result)]
    
    # Booking Tools
    elif name == "search_flights":
        result = search_flights(arguments.get("origin", ""), arguments.get("destination", ""), arguments.get("date", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_flight":
        result = book_flight(arguments.get("flight_id", ""), arguments.get("passenger_name", ""))
        return [TextContent(type="text", text=result)]
    elif name == "search_hotels":
        result = search_hotels(arguments.get("city", ""), arguments.get("check_in", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_hotel":
        result = book_hotel(arguments.get("hotel_id", ""), arguments.get("guest_name", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_ride":
        result = book_ride(arguments.get("pickup", ""), arguments.get("dropoff", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_activity":
        result = book_activity(arguments.get("activity_name", ""), arguments.get("date", ""))
        return [TextContent(type="text", text=result)]
    
    # Utility Tools
    elif name == "get_weather_forecast":
        result = get_weather_forecast(arguments.get("city", ""))
        return [TextContent(type="text", text=result)]
    elif name == "convert_currency":
        result = convert_currency(arguments.get("amount", 0.0), arguments.get("from_curr", ""), arguments.get("to_curr", ""))
        return [TextContent(type="text", text=result)]
    elif name == "translate_text":
        result = translate_text(arguments.get("text", ""), arguments.get("target_lang", ""))
        return [TextContent(type="text", text=result)]
    elif name == "check_visa_requirements":
        result = check_visa_requirements(arguments.get("citizenship", ""), arguments.get("country", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_insurance_quote":
        result = get_insurance_quote(arguments.get("destination", ""), arguments.get("days", 1))
        return [TextContent(type="text", text=result)]
    elif name == "get_emergency_contacts":
        result = get_emergency_contacts(arguments.get("city", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_flight_status":
        result = get_flight_status(arguments.get("flight_number", ""))
        return [TextContent(type="text", text=result)]
    elif name == "track_expense":
        result = track_expense(arguments.get("item", ""), arguments.get("amount", 0.0))
        return [TextContent(type="text", text=result)]
    elif name == "get_budget_summary":
        result = get_budget_summary()
        return [TextContent(type="text", text=result)]
    
    # Social Tools
    elif name == "update_user_preference":
        result = update_user_preference(arguments.get("key", ""), arguments.get("value", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_user_preferences":
        result = get_user_preferences()
        return [TextContent(type="text", text=result)]
    elif name == "submit_feedback":
        result = submit_feedback(arguments.get("rating", 0), arguments.get("comment", ""))
        return [TextContent(type="text", text=result)]
    elif name == "share_to_social_media":
        result = share_to_social_media(arguments.get("platform", ""), arguments.get("content", ""))
        return [TextContent(type="text", text=result)]
    
    else:
        raise ValueError(f"Unknown tool: {name}")

print("✅ MCP tool handler registered (23 tools)")


✅ MCP tool handler registered (23 tools)


In [12]:
@mcp_server.list_tools()
async def list_tools() -> list[Tool]:
    """List all available tools with schemas"""
    return [
        # Planning Tools (3)
        Tool(name="suggest_destinations", description="Suggests travel destinations",
             inputSchema={"type": "object", "properties": {
                 "budget": {"type": "string"}, "season": {"type": "string"}, "interests": {"type": "string"}},
                 "required": ["budget", "season", "interests"]}),
        Tool(name="create_itinerary", description="Creates day-by-day itinerary",
             inputSchema={"type": "object", "properties": {
                 "destination": {"type": "string"}, "days": {"type": "integer"}},
                 "required": ["destination", "days"]}),
        Tool(name="suggest_activities", description="Suggests activities in a city",
             inputSchema={"type": "object", "properties": {
                 "city": {"type": "string"}, "interests": {"type": "string"}},
                 "required": ["city", "interests"]}),
        
        # Booking Tools (6)
        Tool(name="search_flights", description="Searches for flights",
             inputSchema={"type": "object", "properties": {
                 "origin": {"type": "string"}, "destination": {"type": "string"}, "date": {"type": "string"}},
                 "required": ["origin", "destination", "date"]}),
        Tool(name="book_flight", description="Books a flight",
             inputSchema={"type": "object", "properties": {
                 "flight_id": {"type": "string"}, "passenger_name": {"type": "string"}},
                 "required": ["flight_id", "passenger_name"]}),
        Tool(name="search_hotels", description="Searches for hotels",
             inputSchema={"type": "object", "properties": {
                 "city": {"type": "string"}, "check_in": {"type": "string"}},
                 "required": ["city", "check_in"]}),
        Tool(name="book_hotel", description="Books a hotel",
             inputSchema={"type": "object", "properties": {
                 "hotel_id": {"type": "string"}, "guest_name": {"type": "string"}},
                 "required": ["hotel_id", "guest_name"]}),
        Tool(name="book_ride", description="Books local transportation",
             inputSchema={"type": "object", "properties": {
                 "pickup": {"type": "string"}, "dropoff": {"type": "string"}},
                 "required": ["pickup", "dropoff"]}),
        Tool(name="book_activity", description="Books an activity",
             inputSchema={"type": "object", "properties": {
                 "activity_name": {"type": "string"}, "date": {"type": "string"}},
                 "required": ["activity_name", "date"]}),
        
        # Utility Tools (9)
        Tool(name="get_weather_forecast", description="Gets weather forecast",
             inputSchema={"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}),
        Tool(name="convert_currency", description="Converts currency",
             inputSchema={"type": "object", "properties": {
                 "amount": {"type": "number"}, "from_curr": {"type": "string"}, "to_curr": {"type": "string"}},
                 "required": ["amount", "from_curr", "to_curr"]}),
        Tool(name="translate_text", description="Translates text",
             inputSchema={"type": "object", "properties": {
                 "text": {"type": "string"}, "target_lang": {"type": "string"}},
                 "required": ["text", "target_lang"]}),
        Tool(name="check_visa_requirements", description="Checks visa requirements",
             inputSchema={"type": "object", "properties": {
                 "citizenship": {"type": "string"}, "country": {"type": "string"}},
                 "required": ["citizenship", "country"]}),
        Tool(name="get_insurance_quote", description="Gets insurance quote",
             inputSchema={"type": "object", "properties": {
                 "destination": {"type": "string"}, "days": {"type": "integer"}},
                 "required": ["destination", "days"]}),
        Tool(name="get_emergency_contacts", description="Gets emergency contacts",
             inputSchema={"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}),
        Tool(name="get_flight_status", description="Checks flight status",
             inputSchema={"type": "object", "properties": {"flight_number": {"type": "string"}}, "required": ["flight_number"]}),
        Tool(name="track_expense", description="Logs an expense",
             inputSchema={"type": "object", "properties": {
                 "item": {"type": "string"}, "amount": {"type": "number"}},
                 "required": ["item", "amount"]}),
        Tool(name="get_budget_summary", description="Gets budget summary",
             inputSchema={"type": "object", "properties": {}}),
        
        # Social Tools (4)
        Tool(name="update_user_preference", description="Updates user preference",
             inputSchema={"type": "object", "properties": {
                 "key": {"type": "string"}, "value": {"type": "string"}},
                 "required": ["key", "value"]}),
        Tool(name="get_user_preferences", description="Gets user preferences",
             inputSchema={"type": "object", "properties": {}}),
        Tool(name="submit_feedback", description="Submits feedback",
             inputSchema={"type": "object", "properties": {
                 "rating": {"type": "integer"}, "comment": {"type": "string"}},
                 "required": ["rating", "comment"]}),
        Tool(name="share_to_social_media", description="Shares to social media",
             inputSchema={"type": "object", "properties": {
                 "platform": {"type": "string"}, "content": {"type": "string"}},
                 "required": ["platform", "content"]}),
    ]

print("✅ MCP tool schemas registered (23 tools total)")


✅ MCP tool schemas registered (23 tools total)


### Using the MCP Server

The MCP server is now configured. To use it:

**Option 1: Export and run standalone**
```python
# Save MCP server code to mcp_server.py and run:
# python mcp_server.py
```

**Option 2: Use with ADK (shown below)**

The agents below use FunctionTool for simplicity. For production, consider using McpToolset.


## Running the Concierge Agent with ADK GUI



After executing all cells above, you can access the agent through the ADK web interface:



### Option 1: Using app.py (Recommended)

```bash

# In terminal, run:

python -m google.adk.cli web

```



Then open your browser to the URL shown (typically http://localhost:8000)



### Option 2: Direct from Notebook

If you want to create an app directly in this notebook, run the cell below.

## Interactive Chat in Notebook

Execute the cell below to chat with your Concierge Agent directly in this notebook.

**Note**: For web UI access, the `app.py` file in this directory is already configured. Just run:
```bash
python -m google.adk.cli web
```
Then open http://127.0.0.1:8000


In [ ]:
# Launch ADK Web Interface (Custom Script)
import subprocess
import sys
import time
import webbrowser
import threading
import os
from pathlib import Path

def run_adk_server():
    print("🚀 Preparing ADK Web Server...")
    
    # Create the custom runner script
    runner_script = Path("run_gui_custom.py")
    
    with open(runner_script, "w", encoding="utf-8") as f:
        f.write('''
import sys
import os
from pathlib import Path
import uvicorn

# Add current directory to path
sys.path.insert(0, str(Path(".").absolute()))

try:
    # Import correct function
    from google.adk.cli.fast_api import get_fast_api_app
    print("✅ Imported get_fast_api_app")
except ImportError as e:
    print(f"❌ Failed to import get_fast_api_app: {e}")
    sys.exit(1)

if __name__ == "__main__":
    print("🚀 Starting Fixed ADK Server...")
    
    try:
        # Create web app using the correct API
        # We point agents_dir to current directory so it finds app.py
        app = get_fast_api_app(
            agents_dir=".",
            web=True,
            host="127.0.0.1",
            port=8000
        )
        print("✅ App created successfully")
        
        print("   URL: http://127.0.0.1:8000")
        
        # Run with uvicorn
        uvicorn.run(app, host="127.0.0.1", port=8000)
        
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        import traceback
        traceback.print_exc()
''')
    
    print(f"   Created runner: {runner_script}")
    print("   URL: http://127.0.0.1:8000")
    
    # Open browser automatically
    def open_browser():
        time.sleep(4)
        print("   Opening browser...")
        webbrowser.open("http://127.0.0.1:8000")
    
    threading.Thread(target=open_browser, daemon=True).start()
    
    # Run the custom script
    cmd = [sys.executable, "run_gui_custom.py"]
    
    try:
        # Check port
        import socket
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', 8000)) == 0:
                print("\n⚠️ Port 8000 seems to be in use!")
                print("   Please stop other running servers first.")
                return

        print(f"   Executing: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        
    except KeyboardInterrupt:
        print("\n✅ Server stopped by user.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if runner_script.exists():
            try:
                runner_script.unlink()
                print(f"   Cleaned up {runner_script}")
            except:
                pass

if __name__ == "__main__":
    run_adk_server()


🚀 Preparing ADK Web Server...
   Created runner: run_gui_custom.py
   URL: http://127.0.0.1:8000
   Executing: c:\Users\sande\AppData\Local\Programs\Python\Python311\python.exe run_gui_custom.py
   Opening browser...
